# Using LangChain Agent for analysis of Financial statements

Financial statements of World Health Organization has been used.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

# Verify that the API key is loaded
if not openai_api_key:
    raise ValueError("OpenAI API key not found in environment variables.")

## Extract Text from PDFs

In [2]:
from PyPDF2 import PdfReader
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Extract text from all PDFs
fs_2021 = extract_text_from_pdf("data/fs_2021.pdf")
fs_2022 = extract_text_from_pdf("data/fs_2022.pdf")
fs_2023 = extract_text_from_pdf("data/fs_2023.pdf")
basic_understanding = extract_text_from_pdf("data/basic-understanding-of-a-companys-financials.pdf")

In [3]:
print(f"Length of fs_2021={len(fs_2021)}, fs_2022={len(fs_2022)}, fs_2023={len(fs_2023)}")

Length of fs_2021=203929, fs_2022=248956, fs_2023=256493


## Split Text into Sentences and Words

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # tried 300, 500, 1000
    chunk_overlap=200,  # Overlap to maintain context
    separators=["\n\n", "\n", "\. ", " ", ""],
    length_function=len
)

# Split the text
fs_2021_chunks = text_splitter.split_text(fs_2021)
fs_2022_chunks = text_splitter.split_text(fs_2022)
fs_2023_chunks = text_splitter.split_text(fs_2023)
basic_understanding_chunks = text_splitter.split_text(basic_understanding)

In [5]:
print(f"Length of fs_2021={len(fs_2021)}, fs_2022={len(fs_2022)}, fs_2023={len(fs_2023)}")

Length of fs_2021=203929, fs_2022=248956, fs_2023=256493


## Tokenize the text

In [6]:
# Tokenization is handled internally by LangChain and OpenAI embeddings,

## Create Context-Aware and Regular Word Embeddings

In [7]:
import pickle
import os
from langchain_openai import OpenAIEmbeddings

# Define paths for saving pickled embeddings
fs_2021_pickle_path = "data/fs_2021_embeddings.pkl"
fs_2022_pickle_path = "data/fs_2022_embeddings.pkl"
fs_2023_pickle_path = "data/fs_2023_embeddings.pkl"
basic_understanding_pickle_path = "basic_understanding_embeddings.pkl"

# Function to save embeddings to a pickle file
def save_embeddings_to_pickle(embeddings, file_path):
    with open(file_path, "wb") as f:
        pickle.dump(embeddings, f)

# Function to load embeddings from a pickle file
def load_embeddings_from_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

# Check if pickled embeddings already exist
if os.path.exists(fs_2021_pickle_path) and os.path.exists(fs_2022_pickle_path) and os.path.exists(fs_2023_pickle_path) and os.path.exists(basic_understanding_pickle_path):
    print("Loading embeddings from pickle files...")
    fs_2021_embeddings = load_embeddings_from_pickle(fs_2021_pickle_path)
    fs_2022_embeddings = load_embeddings_from_pickle(fs_2022_pickle_path)
    fs_2023_embeddings = load_embeddings_from_pickle(fs_2023_pickle_path)
    basic_understanding_embeddings = load_embeddings_from_pickle(basic_understanding_pickle_path)
else:
    print("Generating embeddings and saving to pickle files...")
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

    # Generate embeddings for each chunk
    fs_2021_embeddings = embeddings.embed_documents(fs_2021_chunks)
    fs_2022_embeddings = embeddings.embed_documents(fs_2022_chunks)
    fs_2023_embeddings = embeddings.embed_documents(fs_2023_chunks)
    basic_understanding_embeddings = embeddings.embed_documents(basic_understanding_chunks)

    # Save embeddings to pickle files
    save_embeddings_to_pickle(fs_2021_embeddings, fs_2021_pickle_path)
    save_embeddings_to_pickle(fs_2022_embeddings, fs_2022_pickle_path)
    save_embeddings_to_pickle(fs_2023_embeddings, fs_2023_pickle_path)
    save_embeddings_to_pickle(basic_understanding_embeddings, basic_understanding_pickle_path)

Generating embeddings and saving to pickle files...


## TF-IDF and SVM Retriever

In [9]:
from langchain.retrievers import SVMRetriever, TFIDFRetriever
splits = fs_2021_chunks + fs_2022_chunks + fs_2023_chunks + basic_understanding_chunks

svm_retriever = SVMRetriever.from_texts(splits, embeddings)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

question = "What are major highlights in each of the financial reports from 2021, 2022, and 2023?"

docs_svm = svm_retriever.invoke(question)
docs_tfidf = tfidf_retriever.invoke(question)


In [10]:
# Print SVMRetriever results
print("SVMRetriever Results:")
for doc in docs_svm:
    print(doc.page_content)
    print("-" * 50)

SVMRetriever Results:
The benefits of an annual report
The annual report contains a significant amount of information:
Financial
• Management discussion & 
analysis(MD&A)
• Financial statements
• Notes to financial statements• Messages from the Chair, CEO
• Corporate profile 
• MD&A
• Risk and control processes and analysisNon-financialOperational 
performanceFinancial 
performance
Strategic 
direction
Source CFI33 PwC | Basic Understanding of a Company's FinancialsLetters to the 
shareholders01.
Business description02.
Management’s 
Discussion and 
Analysis (MD&A)03.
Reporting on internal 
controls04.
Audit report05.
Balance sheet, 
Income Statement 
and Statement of 
Cash Flows06.
Notes to the financial 
statements07.
Earnings per share08.
Earnings per share
Listing of directors of 
the company09.Contents of an annual report
The annual report will always include:
Source CFI34 PwC | Basic Understanding of a Company's FinancialsActs as sort of variance 
analysisExplains company 
perfor

In [11]:
# Print TFIDFRetriever results
print("TFIDFRetriever Results:")
for doc in docs_tfidf:
    print(doc.page_content)
    print("-" * 50)

TFIDFRetriever Results:
Organization is in the process of developing  corporate -wide actions to address these points and advance 
its overall risk maturity.  
(c) Reports issued by the Office of Internal Oversight. The internal audit reports, conducted under the 
audit workplan  for 2021  and preceding years , provide objective information on compliance and control 
effectiveness, together with recommendations for improvement. Critical findings from these reports are 
summarized in the Annual Report of the Internal Auditor to the Health Assembly. Individual audit reports 
are available for review by Member States on request.  
(d) Reports issued by the WHO External Auditor. The external audit provides independent oversight 
and reporting on WHO’s compliance with financial rules and regulations. The external auditors present A75/33 
 
 
 
 
 
 
20 their key findings, observations and recommendations to the governing bodies. In addition, t hey also have
---------------------------------

## Update ChromaDB Client Initialization

In [12]:
import chromadb
from chromadb.config import Settings
print("ChromaDB and Settings imported successfully!")

client = chromadb.PersistentClient(path="./chroma_db")

# Create or load a collection
collection = client.get_or_create_collection(name="financial_statements")

print("Collection created successfully!")

ChromaDB and Settings imported successfully!
Collection created successfully!


##  Store Embeddings in ChromaDB

In [13]:
def add_documents_to_collection(collection, chunks, embeddings, metadata, id_prefix):
    """
    Add documents to the collection while avoiding duplicates.
    
    Args:
        collection: The ChromaDB collection.
        chunks: List of text chunks.
        embeddings: List of embeddings corresponding to the chunks.
        metadata: List of metadata dictionaries.
        id_prefix: Prefix for the document IDs.
    """
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        doc_id = f"{id_prefix}_{i}"
        
        # Check if the document already exists in the collection
        existing_docs = collection.get(ids=[doc_id])
        
        # If the document does not exist, add it
        if not existing_docs["ids"]:
            collection.add(
                documents=[chunk],
                embeddings=[embedding],
                metadatas=[metadata[i]],
                ids=[doc_id]
            )
            print(f"Added document with ID: {doc_id}")
        else:
            print(f"Document with ID {doc_id} already exists. Skipping.")

# Add documents to the collection while avoiding duplicates
add_documents_to_collection(collection, fs_2021_chunks, fs_2021_embeddings, [{"year": 2021, "source": "fs_2021.pdf"}] * len(fs_2021_chunks), "fs_2021")
add_documents_to_collection(collection, fs_2022_chunks, fs_2022_embeddings, [{"year": 2022, "source": "fs_2022.pdf"}] * len(fs_2022_chunks), "fs_2022")
add_documents_to_collection(collection, fs_2023_chunks, fs_2023_embeddings, [{"year": 2023, "source": "fs_2023.pdf"}] * len(fs_2023_chunks), "fs_2023")
add_documents_to_collection(collection, basic_understanding_chunks, basic_understanding_embeddings, [{"source": "basic-understanding-of-a-companys-financials.pdf"}] * len(basic_understanding_chunks), "basic_understanding")

Added document with ID: fs_2021_0
Added document with ID: fs_2021_1
Added document with ID: fs_2021_2
Added document with ID: fs_2021_3
Added document with ID: fs_2021_4
Added document with ID: fs_2021_5
Added document with ID: fs_2021_6
Added document with ID: fs_2021_7
Added document with ID: fs_2021_8
Added document with ID: fs_2021_9
Added document with ID: fs_2021_10
Added document with ID: fs_2021_11
Added document with ID: fs_2021_12
Added document with ID: fs_2021_13
Added document with ID: fs_2021_14
Added document with ID: fs_2021_15
Added document with ID: fs_2021_16
Added document with ID: fs_2021_17
Added document with ID: fs_2021_18
Added document with ID: fs_2021_19
Added document with ID: fs_2021_20
Added document with ID: fs_2021_21
Added document with ID: fs_2021_22
Added document with ID: fs_2021_23
Added document with ID: fs_2021_24
Added document with ID: fs_2021_25
Added document with ID: fs_2021_26
Added document with ID: fs_2021_27
Added document with ID: fs_202

## Verify ChromaDB Population

In [14]:
# Query the Collection
# Retrieve all items from the collection
items = collection.get()

In [15]:
# Inspect the keys in the returned dictionary
print("Keys in the collection:", items.keys())

Keys in the collection: dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])


In [16]:
# Print the first few documents and their metadata
for doc, metadata in zip(items["documents"][:5], items["metadatas"][:5]):
    print("Document:", doc)
    print("Metadata:", metadata)
    print("-" * 50)

Document: AUDITED FINANCIAL STATEMENTS  
FOR THE YEAR ENDED 31 DECEMBER 2021A75/33  
SEVENTY-FIFTH WORLD HEALTH ASSEMBLY  
Provisional agenda item 22.1  
13 May 2022Table of contents
Financial statements at a glance        2
Director-General’s summary          4
2021 Statement of Internal Control        18
Certification of financial statements for the year ended 31 December 2021      22
Opinion of the External Auditor        23
Independent Auditor’s report         25
Financial statements          28
 Statement I. Statement of Financial Position      28
 Statement II. Statement of Financial Performance      29
 Statement III. Statement of Changes in Net Assets/Equity     30
 Statement IV. Statement of Cash Flow        31
 Statement V. Statement of Comparison of Budget and Actual Amounts  32
1. Notes to the financial statements         33
2. Significant accounting policies        34
3. Note on the restatement/reclassification of balances     44
Metadata: {'source': 'fs_2021.pdf', 'year':

In [17]:
# Check the Number of Items
print("Number of items in the collection:", len(items["ids"]))

Number of items in the collection: 900


## Similarity Search

In [25]:
# Define the question
question1 = "List the total assets and liabilities for each of the years 2021, 2022, and 2023"

# Initialize the vector store
from langchain_chroma import Chroma
vector_store = Chroma(
    client=client,
    collection_name="financial_statements",
    embedding_function=embeddings
)

result1 = vector_store.similarity_search(question1, k=5) # Retrieve top 5 relevant chunks
for doc in result1:
    print(doc.page_content)
    print("-" * 50)


Schedule III. Financial overview – all funds, 2022, 2020–2021 and 2018–2019  103
Annex I. Revenue and expenses with programme countries 104
Annex II. Revenue and expenses with United Nations entities 108A76/172
Financial Statements 2022 at a Glance
REVENUE
2022: US$ 4354 m 
2021: US$ 4066 mEXPENSES
2022: US$ 3848 m 
2021: US$ 3718 mSURPLUS/(DEFICIT)1
2022: US$ 600 m 
2021: US$ 351 m
1 Includes Finance Revenue (2022: US$ 94m, 
2021: US$ 3m)
ASSETS
2022: US$ 7812 m 
2021: US$ 7214LIABILITIES
2022: US$ 2792 m 
2021: US$ 3807 mNET ASSETS/EQUITY
2022: US$ 5020 m 
2021: US$ 3407 mNon-Programme budget (PB)/
other revenueVoluntary contributions (PB)Assessed contributions (PB) Staff costs
Opening Net AssetsStaff related
Financial AssetsMedical supplies and materials
Surplus/(deficit)Inter-entity
Fixed assetsContractual services
Other adjustmentsOther
ReceivablesTransfers and grants
Borrowing
OtherTravel
Deferred and advance contributionsGeneral operating expenses
-------------------------------

In [26]:
len(result1)

5

In [27]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4", temperature=0)

# Combine the retrieved results into a single context
context = "\n".join([doc.page_content for doc in result1])

# Define a prompt to extract assets and liabilities
prompt = f"""
You are a financial analyst. Use the following financial data to extract the total assets and liabilities for the years 2021, 2022, and 2023:

{context}

Format the results as follows:
- Year: [Year]
  - Total Assets: [Amount]
  - Total Liabilities: [Amount]
"""

# Generate the response
response1 = llm.invoke(prompt)
print(response1.content)


- Year: 2021
  - Total Assets: US$ 7214 m
  - Total Liabilities: US$ 3807 m
- Year: 2022
  - Total Assets: US$ 7812 m
  - Total Liabilities: US$ 2792 m
- Year: 2023
  - Total Assets: US$ 7231 m
  - Total Liabilities: US$ 2699 m


## Max Marginal Relevance Search

In [28]:
# Perform a max marginal relevance search
result2 = vector_store.max_marginal_relevance_search(question1, k=5, fetch_k=3)

# Print the results
for doc in result2:  # Use `results2` instead of `result2`
    print(doc.page_content)
    print("-" * 50)

Schedule III. Financial overview – all funds, 2022, 2020–2021 and 2018–2019  103
Annex I. Revenue and expenses with programme countries 104
Annex II. Revenue and expenses with United Nations entities 108A76/172
Financial Statements 2022 at a Glance
REVENUE
2022: US$ 4354 m 
2021: US$ 4066 mEXPENSES
2022: US$ 3848 m 
2021: US$ 3718 mSURPLUS/(DEFICIT)1
2022: US$ 600 m 
2021: US$ 351 m
1 Includes Finance Revenue (2022: US$ 94m, 
2021: US$ 3m)
ASSETS
2022: US$ 7812 m 
2021: US$ 7214LIABILITIES
2022: US$ 2792 m 
2021: US$ 3807 mNET ASSETS/EQUITY
2022: US$ 5020 m 
2021: US$ 3407 mNon-Programme budget (PB)/
other revenueVoluntary contributions (PB)Assessed contributions (PB) Staff costs
Opening Net AssetsStaff related
Financial AssetsMedical supplies and materials
Surplus/(deficit)Inter-entity
Fixed assetsContractual services
Other adjustmentsOther
ReceivablesTransfers and grants
Borrowing
OtherTravel
Deferred and advance contributionsGeneral operating expenses
-------------------------------

In [29]:
# Generate the response

# Combine the retrieved results into a single context
context = "\n".join([doc.page_content for doc in result2])

response2 = llm.invoke(prompt)
print(response2.content)


- Year: 2021
  - Total Assets: US$ 7214 m
  - Total Liabilities: US$ 3807 m
- Year: 2022
  - Total Assets: US$ 7812 m
  - Total Liabilities: US$ 2792 m
- Year: 2023
  - Total Assets: US$ 7231 m
  - Total Liabilities: US$ 2699 m


## Using ***compression*** to improve quality of retrieved docs

In [30]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI, ChatOpenAI

llm_compressor = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct", openai_api_key=openai_api_key)
compressor = LLMChainExtractor.from_llm(llm_compressor)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vector_store.as_retriever())

question = "What are major highlights in each of the financial reports from 2021, 2022, and 2023?"

# Retrieve and compress relevant documents
compressed_docs = compression_retriever.get_relevant_documents(question, k=10)  # Increase k if needed

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

pretty_print_docs(compressed_docs)

# Initialize the language model for summarization
llm_summarizer = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4", temperature=0)

# Combine the retrieved results into a single context
context = "\n".join([doc.page_content for doc in compressed_docs])

# Define a prompt to extract highlights
prompt = f"""
You are a financial analyst. Use the following financial data to extract the major highlights for the years 2021, 2022, and 2023:

{context}

Format the results as follows:
- Year: [Year]
  - Highlights: [List of highlights]
"""

# Generate the response
response = llm_summarizer.invoke(prompt)
print(response.content)

/tmp/ipykernel_27479/1978046480.py:12: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  compressed_docs = compression_retriever.get_relevant_documents(question, k=10)  # Increase k if needed


Document 1:

Statement I. Statement of Financial Position   40
Statement II. Statement of Financial Performance   
Statement III. Statement of Changes in Net Assets/Equity   42
Statement IV. Statement of Cash Flow   43
Statement V. Statement of Comparison of Budget and Actual Amounts  44
1. Notes to the financial statements 45
2. Significant accounting policies 49
3. Note on the restatement/reclassification of balances 60
----------------------------------------------------------------------------------------------------
Document 2:

2022: US$ 4354 m 
2021: US$ 4066 mEXPENSES
2022: US$ 3848 m 
2021: US$ 3718 mSURPLUS/(DEFICIT)1
2022: US$ 600 m 
2021: US$ 351 m
1 Includes Finance Revenue (2022: US$ 94m, 
2021: US$ 3m)
----------------------------------------------------------------------------------------------------
Document 3:

1. Notes to the financial statements         33
2. Significant accounting policies        34
3. Note on the restatement/reclassification of balances     44
4. 

## Persist the Database

In [14]:
# client.persist()
# AttributeError: 'Client' object has no attribute 'persist'

## Create LangChain Agents and Prompts

In [31]:
from pydantic import BaseModel
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Define the Pydantic model
class QueryResult(BaseModel):
    query: str
    result: str

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Initialize the LLM
llm = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4", temperature=0)

# Create a retrieval-based QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

# Define a prompt template
prompt_template = """
You are a financial analyst. Use the provided financial statements and the document "basic understanding of a company's financials" to answer the following question:
Separate the answers with line breaks.
Question: {question}
"""

# Wrap the query Output in the Pydantic Model
def query_agent(question: str) -> QueryResult:
    prompt = prompt_template.format(question=question)
    response = qa_chain.invoke(prompt)
    
    # Extract the result from the response dictionary
    result = response.get("result", "")

    return QueryResult(query=prompt, result=result)


## Test the Agent

In [32]:
from IPython.display import display, HTML

In [33]:
# Example query
question1 = "What are the key trends in the company's revenue from 2021 to 2023?"
answer1 = query_agent(question1)
print(answer1)

query='\nYou are a financial analyst. Use the provided financial statements and the document "basic understanding of a company\'s financials" to answer the following question:\nSeparate the answers with line breaks.\nQuestion: What are the key trends in the company\'s revenue from 2021 to 2023?\n' result="The key trends in the company's revenue from 2021 to 2023 are:\n\n- In 2021, the total revenue was US$ 4066 million.\n- In 2022, the total revenue increased to US$ 4354 million, which was an increase of US$ 288 million compared to 2021.\n- However, in 2023, the total revenue declined to US$ 3341 million, a decrease of US$ 1013 million compared to 2022. The main drivers of this decrease were the reduction in voluntary contributions by US$ 911 million and the reduction in in-kind contributions by US$ 91 million."


In [34]:
# display(HTML(str(answer1)))

In [35]:
question2 = "Explain the trends in the organization's cash flow statement from 2021 to 2022 based on the basic understanding document?"
answer2 = query_agent(question2)
print(answer2)

query='\nYou are a financial analyst. Use the provided financial statements and the document "basic understanding of a company\'s financials" to answer the following question:\nSeparate the answers with line breaks.\nQuestion: Explain the trends in the organization\'s cash flow statement from 2021 to 2022 based on the basic understanding document?\n' result="I'm sorry, but I can't provide the information you're looking for because the cash flow statement for the organization from 2021 to 2022 hasn't been provided in the context."


In [36]:
question3 = "How has the cash and assets changed over the years?"
answer3 = query_agent(question3)
print(answer3)

query='\nYou are a financial analyst. Use the provided financial statements and the document "basic understanding of a company\'s financials" to answer the following question:\nSeparate the answers with line breaks.\nQuestion: How has the cash and assets changed over the years?\n' result="I'm sorry, but I can't provide the information you're looking for. The documents provided do not include historical data or multiple years of financial statements, which are necessary to determine how cash and assets have changed over the years."


In [37]:
question4 = "How has the contribution to the revenue changed over the years?"
answer4 = query_agent(question4)
print(answer4)

query='\nYou are a financial analyst. Use the provided financial statements and the document "basic understanding of a company\'s financials" to answer the following question:\nSeparate the answers with line breaks.\nQuestion: How has the contribution to the revenue changed over the years?\n' result='The financial statement provided shows the revenue contributions for the years 2020 and 2021. \n\nIn 2021, the assessed contributions were 549,293 (in US$ thousands), which is an increase from 465,946 in 2020. \n\nVoluntary contributions decreased from 3,704,226 in 2020 to 3,365,228 in 2021. \n\nVoluntary contributions in-kind and in-service increased from 79,712 in 2020 to 93,123 in 2021. \n\nOther revenue also increased from 49,450 in 2020 to 58,671 in 2021. \n\nOverall, the total revenue decreased from 4,299,334 in 2020 to 4,066,315 in 2021.'
